In [ ]:
# Imports all necessary libraries for building, training, evaluating, and visualizing a convolutional neural network for multiclass image classification
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Rescaling, Flatten
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from matplotlib.pyplot import imshow
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import os
import pandas as pd

In [ ]:
# Displays an example image from the dataset to determine the input size for the neural network
%matplotlib inline
pil_im = Image.open('/kaggle/input/alzheimers-mri-images/Processed Dataset/training/Non Demented/26 (100).jpg', 'r')
imshow(np.asarray(pil_im))

In [ ]:
# Loads the training, validation, and test datasets from directories, sets image size and batch size, and optimizes data loading with prefetching
image_size = (100, 100)  # Images of 100x100 dimension
batch_size = 32          # Number of images that are passed to the network per epoch

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/alzheimers-mri-images/Processed Dataset/training",
    validation_split = 0.2,     # 20% of images for validation
    subset = "training",
    seed = 1337,                # In an iteration, we do not feed the network with all the images but with small portions given by the batch_size
    image_size = image_size,    # Size of our dataset images
    batch_size = batch_size,
    label_mode = 'categorical'  # The network classifies more than two classes (categorical), if the network would classify two classes (binary)
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/alzheimers-mri-images/Processed Dataset/validation",
    validation_split = 0.2,     # 80% of images for training
    subset = "validation",
    seed = 1337,                # This is a specific way of randomizing the data used during the training phase of the network
    image_size = image_size,    # Recommended images sizes (100, 200, 512)
    batch_size = batch_size,
    label_mode = 'categorical'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/alzheimers-mri-images/Processed Dataset/test",
    validation_split = 0.2,     # 80% of images for training
    subset = "training",
    seed = 1337,                # This is a specific way of randomizing the data used during the training phase of the network
    image_size = image_size,    # Recommended images sizes (100, 200, 512)
    batch_size = batch_size,
    label_mode = 'categorical'
)

train_ds = train_ds.prefetch(buffer_size = 32)
val_ds = val_ds.prefetch(buffer_size = 32)
test_ds = test_ds.prefetch(buffer_size = 32)

In [ ]:
# Calculates and visualizes the number of images in each class within the training dataset
train_dir = "/kaggle/input/alzheimers-mri-images/Processed Dataset/training"

# Counts the number of images in each class
class_counts = {cls: len(os.listdir(os.path.join(train_dir, cls))) for cls in os.listdir(train_dir)}

# Converts to DataFrame
df = pd.DataFrame(list(class_counts.items()), columns=["Class", "Count"])

# Plots the number of images per class
plt.figure(figsize = (15, 8))
ax = sns.barplot(x = df["Class"], y = df["Count"], palette = "Set1")
ax.set_xlabel("Class", fontsize = 20)
ax.set_ylabel("Count", fontsize = 20)
plt.title("The Number Of Samples For Each Class", fontsize = 20)
plt.grid(True)
plt.xticks(rotation = 45)
plt.show()

In [ ]:
# Initializes a sequential model for building the CNN architecture
model = keras.Sequential()

# Builds the convolutional neural network model
model.add(Rescaling(scale = (1. / 127.5),  # Layer for the normalization of the dataset image set
                    offset = -1,
                    input_shape = (100, 100, 3)))

model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(512, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.7))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(4, activation = 'softmax')

In [ ]:
# Specifies the model compilation mode
model.compile(loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(1e-3),
    metrics = ['accuracy']
)

# Displays a summary and a visual diagram of the model architecture
model.summary()

tf.keras.utils.plot_model(model, to_file = 'model.png', show_shapes = True, show_layer_names = True, show_dtype = True, dpi = 120)

In [ ]:
# Trains the model using the training and validation datasets, applying early stopping to prevent overfitting
epochs = 200

# Once the highest value is reached in the network and it stabilizes or starts
# to have losses, the training process is stopped. The parameter patience=10 makes
# it stop after 10 waiting epochs when the highest value has been recorded and the
# model has not improved, so that finally we will have the best value and therefore
# the best weights
es = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 10, restore_best_weights = True)

# Fits the model given training and validation data, the number of epochs and each of the previously implemented layers
h = model.fit(
        train_ds,
        epochs = epochs,
        validation_data = val_ds,
        callbacks = [es]
)

In [ ]:
# Graphical representation of the results obtained during the training phase
plt.plot(h.history['accuracy'])
plt.plot(h.history['val_accuracy'])
plt.plot(h.history['loss'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation','loss'], loc='upper right')
plt.show()

In [ ]:
# Concatenates true labels and model predictions for all batches in the validation set
results = np.concatenate([(y, model.predict(x = x)) for x, y in val_ds], axis = 1)

# Gets predicted class indices from model outputs
predictions = np.argmax(results[0], axis = 1)

# Gets true class indices from one-hot encoded labels
labels = np.argmax(results[1], axis = 1)

# Computes the confusion matrix
cf_matrix = confusion_matrix(labels, predictions)

# Plots the confusion matrix as a heatmap
sns.heatmap(cf_matrix, annot = True, fmt = "d", cmap = "Blues")

# Prints a detailed classification report
print(classification_report(labels, predictions, digits = 4))

In [ ]:
# Concatenates true labels and model predictions for all batches in the validation set
results = np.concatenate([(y, model.predict(x = x)) for x, y in val_ds], axis = 1)

# Gets predicted class indices from model outputs
predictions = np.argmax(results[0], axis = 1)

# Gets true class indices from one-hot encoded labels
labels = np.argmax(results[1], axis = 1)

# Computes the confusion matrix
cf_matrix = confusion_matrix(labels, predictions)

# Plots the confusion matrix as a heatmap
sns.heatmap(cf_matrix, annot = True, fmt = "d", cmap = "Blues")

# Prints a detailed classification report
print(classification_report(labels, predictions, digits = 4))

In [ ]:
# Concatenates true labels and model predictions for all batches in the validation set
results = np.concatenate([(y, model.predict(x = x)) for x, y in val_ds], axis = 1)

# Gets predicted class indices from model outputs
predictions = np.argmax(results[0], axis = 1)

# Gets true class indices from one-hot encoded labels
labels = np.argmax(results[1], axis = 1)

# Computes the confusion matrix
cf_matrix = confusion_matrix(labels, predictions)

# Plots the confusion matrix as a heatmap
sns.heatmap(cf_matrix, annot = True, fmt = "d", cmap = "Blues")

# Prints a detailed classification report
print(classification_report(labels, predictions, digits = 4))

In [ ]:
# Defines class names for Alzheimer's MRI dataset
class_names = ['Mild Demented', 'Moderate Demented', "Non Demented", "Very Mild Demented"]

# Generates predictions for the test dataset
predictions = model.predict(test_ds)

y_pred = np.argmax(predictions, axis = 1)
y_real = np.concatenate([y for x, y in test_ds], axis = 0)
y_real = np.argmax(y_real, axis = 1)  # Converts one-hot labels to class indices

for p, l in zip(predictions, y_real):
    probs_percent = [f"{prob*100:.2f}%" for prob in p]  # Converts probabilities to percentage
    predicted_class_idx = np.argmax(p)  # Index of the predicted class
    predicted_class_name = class_names[predicted_class_idx]  # Name of the predicted class

    print(f"Predictions: {probs_percent} -> Predicted class: {predicted_class_name} (Class {predicted_class_idx}), Actual Label: {class_names[l]}")

    if predicted_class_idx == l:
        print("Correct ✅\n")
    else:
        print("Incorrect ❌\n")